In [1]:
%load_ext autoreload

from ipywidgets import interact, interactive, IntSlider, Layout, RadioButtons
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

import pandas as pd
from ipywidgets import HBox, Label, FloatSlider

style = {'description_width': 'initial'}

def plot_random_walk(n):
    np.random.seed(0)
    n = int(n)
    x = np.linspace(0.0, 10.0, n)
    
    y = np.cumsum(np.random.randn(n))
    print(y[-10:])
    f, ax = plt.subplots(figsize=(8, 6))
    # plt.figure(3, title='Line Chart')
    # plt.figure()
    ax.plot(x, y)
    plt.show()

lb1=Label (value="Play with options and watch the probability space evolve dynamically. \
Remember, smaller the value of $C$, stronger the regularization strength",fontsize=15)

logp=interactive(plot_random_walk,n=FloatSlider(value=10, min=0, max=1000, step=10,
                                       description='number of dots', style=style, continuous_update=True))

# Set the height of the control.children[-1] so that the output does not jump and flicker
output = logp.children[-1]
output.layout.height = '500px'
output.layout.width = '1000px'

display(lb1)
display(logp)

Label(value='Play with options and watch the probability space evolve dynamically. Remember, smaller the value of $C$, stronger the regularization strength')

interactive(children=(FloatSlider(value=10.0, description='number of dots', max=1000.0, step=10.0, style=SliderStyle(description_width='initial')), Output(layout=Layout(height='500px', width='1000px'))), _dom_classes=('widget-interact',))

In [2]:
import sys
if "'/Users/martin/Projects/tick'" in sys.path:
    sys.path.remove('/Users/martin/Projects/tick')
sys.path = ['/Users/martin/Projects/tick_expe'] + sys.path


In [11]:
from poisson_see_duals import load_dataset, run_solver, load_experiments
from tick.optim.model import ModelPoisReg
from scipy.stats import linregress
from tick.plot import plot_history

import statsmodels.api as sm
import numpy as np

def plot_dual_vs_init(n_samples, n_features, l_l2sq_power, max_iter, init_type):
    l_l2sq = np.power(n_samples, l_l2sq_power)
    dataset = 'simulated_{}_{}'.format(n_samples, n_features)
    features, labels = load_dataset(dataset)
    
    solver = run_solver(dataset, features, labels, l_l2sq, max_iter_sdca=max_iter)
    
    model = ModelPoisReg(fit_intercept=False, link='identity')
    model.fit(features, labels)
    
    duals = load_experiments(dataset, l_l2sq, 'duals')
    print('init_type', init_type)
    dual_inits = model.get_dual_init(l_l2sq, init_type=init_type)
    
    non_zero_features = features[labels != 0]
    norms = np.linalg.norm(non_zero_features, axis=1)
    normalized_inits = dual_inits / norms
    normalized_duals = duals / norms
    
    slope, intercept, r_value, p_value, std_err = \
        linregress(normalized_inits, normalized_duals)
    
    f, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(normalized_inits, normalized_duals)
    ax.plot([0, max(normalized_inits)],
                 [intercept, intercept + slope * max(normalized_inits)],
                 color='C1', label='{:.2g} + {:.4g}x'.format(intercept, slope))
    
    linreg_no_interceot = sm.OLS(normalized_duals, normalized_inits)
    slope_no_intercept = linreg_no_interceot.fit().params[0]
    ax.plot([0, max(normalized_inits)],
                 [0, slope_no_intercept * max(normalized_inits)],
                 color='C2', label='{:.4g}x'.format(slope_no_intercept))
    
    ax.plot([0, max(normalized_inits)],
             [0, max(normalized_inits)],
             color='C3', label='x')

    ax.set_ylabel('dual sol / norm')
    ax.set_xlabel('dual init / norm')
    ax.legend()
    plt.show()
    
    f, ax = plt.subplots(figsize=(8, 3))
    solver.history.values['duality_gap'] = [np.abs(x) if np.isfinite(x) else np.nan
                                            for x in solver.history.values['duality_gap']]
    plot_history([solver], log_scale=True, ax=ax, y='duality_gap')

n_samples_slider = IntSlider(value=1000, min=0, max=10000, step=10,
                             description='number of samples', style=style, continuous_update=False)

n_features_slider = IntSlider(value=10, min=0, max=1000, step=1,
                              description='number of features', style=style, continuous_update=False)

max_iter_slider = IntSlider(value=1000, min=10, max=10000, step=10,
                            description='max iterations', style=style, continuous_update=False)

l_l2sq_slider = FloatSlider(value=-0.5, min=-2, max=0, step=0.01,
                            description='Regularization $\\lambda = n^{x}$)', style=style, continuous_update=False)


init_type_radio = RadioButtons(options=['psi', 'features'], description='Init type')
    
dual_vs_init = interactive(plot_dual_vs_init,
                           n_samples=n_samples_slider, n_features=n_features_slider,
                           l_l2sq_power = l_l2sq_slider, max_iter=max_iter_slider,
                          init_type=init_type_radio)

# Set the height of the control.children[-1] so that the output does not jump and flicker
output = dual_vs_init.children[-1]
output.layout.height = '1000px'
output.layout.width = '800px'

display(dual_vs_init)

interactive(children=(IntSlider(value=1000, continuous_update=False, description='number of samples', max=10000, step=10, style=SliderStyle(description_width='initial')), IntSlider(value=10, continuous_update=False, description='number of features', max=1000, style=SliderStyle(description_width='initial')), FloatSlider(value=-0.5, continuous_update=False, description='Regularization $\\lambda = n^{x}$)', max=0.0, min=-2.0, step=0.01, style=SliderStyle(description_width='initial')), IntSlider(value=1000, continuous_update=False, description='max iterations', max=10000, min=10, step=10, style=SliderStyle(description_width='initial')), RadioButtons(description='Init type', options=('psi', 'features'), value='psi'), Output(layout=Layout(height='1000px', width='800px'))), _dom_classes=('widget-interact',))

In [4]:
import tick
tick

<module 'tick' from '/Users/martin/Projects/tick_expe/tick/__init__.py'>